In [32]:
from anthropic.types import Message, TextBlock, ToolUseBlock
from claudette.core import *
from fastcore.meta import delegates
from fastcore.utils import *

In [3]:
model = models[-1]


In [4]:
model

'claude-3-haiku-20240307'

In [5]:
orders = {
    "O1": dict(id="O1", product="Widget A", quantity=2, price=19.99, status="Shipped"),
    "O2": dict(id="O2", product="Gadget B", quantity=1, price=49.99, status="Processing"),
    "O3": dict(id="O3", product="Gadget B", quantity=2, price=49.99, status="Shipped")}

customers = {
    "C1": dict(name="John Doe", email="john@example.com", phone="123-456-7890",
               orders=[orders['O1'], orders['O2']]),
    "C2": dict(name="Jane Smith", email="jane@example.com", phone="987-654-3210",
               orders=[orders['O3']])
}

In [7]:
orders

{'O1': {'id': 'O1',
  'product': 'Widget A',
  'quantity': 2,
  'price': 19.99,
  'status': 'Shipped'},
 'O2': {'id': 'O2',
  'product': 'Gadget B',
  'quantity': 1,
  'price': 49.99,
  'status': 'Processing'},
 'O3': {'id': 'O3',
  'product': 'Gadget B',
  'quantity': 2,
  'price': 49.99,
  'status': 'Shipped'}}

In [8]:
def get_customer_info(
    customer_id:str # ID of the customer
): # Customer's name, email, phone number, and list of orders
    "Retrieves a customer's information and their orders based on the customer ID"
    print(f'- Retrieving customer {customer_id}')
    return customers.get(customer_id, "Customer not found")

def get_order_details(
    order_id:str # ID of the order
): # Order's ID, product name, quantity, price, and order status
    "Retrieves the details of a specific order based on the order ID"
    print(f'- Retrieving order {order_id}')
    return orders.get(order_id, "Order not found")

def cancel_order(
    order_id:str # ID of the order to cancel
)->bool: # True if the cancellation is successful
    "Cancels an order based on the provided order ID"
    print(f'- Cancelling order {order_id}')
    if order_id not in orders: return False
    orders[order_id]['status'] = 'Cancelled'
    return True

In [9]:
tools = [get_customer_info, get_order_details, cancel_order]
chat = Chat(model, tools=tools)

In [10]:
r = chat('Can you tell me the email address for customer C1?')
print(r.stop_reason)
r.content
     

- Retrieving customer C1
tool_use


[ToolUseBlock(id='toolu_01UZ7q7dkbBUEEbFvitUu87L', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]

In [20]:
get_customer_info('C1')

- Retrieving customer C1


{'name': 'John Doe',
 'email': 'john@example.com',
 'phone': '123-456-7890',
 'orders': [{'id': 'O1',
   'product': 'Widget A',
   'quantity': 2,
   'price': 19.99,
   'status': 'Cancelled'},
  {'id': 'O2',
   'product': 'Gadget B',
   'quantity': 1,
   'price': 49.99,
   'status': 'Cancelled'}]}

In [11]:
r = chat()

In [12]:
contents(r)

'The email address for customer C1 is john@example.com.'

In [13]:
chat = Chat(model, tools=tools)

In [14]:
r = chat("Please cancel all orders for customer C1 for me.")

- Retrieving customer C1


In [15]:
r.stop_reason

'tool_use'

In [17]:
r.content

[TextBlock(text="Okay, let's cancel all orders for customer C1:", type='text'),
 ToolUseBlock(id='toolu_01TuVzBRLTecVGux3a3DVVBs', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]

In [18]:

chat = Chat(model, tools=tools)
r = chat.toolloop('Can you tell me the email address for customer C1?')
r

- Retrieving customer C1


The email address for customer C1 is john@example.com.

<details>

- id: msg_01HY1mhR62SWyexjzkzQLrWS
- content: [{'text': 'The email address for customer C1 is john@example.com.', 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 732, 'output_tokens': 19}

</details>

In [44]:
tools

[<function __main__.get_customer_info(customer_id: str)>,
 <function __main__.get_order_details(order_id: str)>,
 <function __main__.cancel_order(order_id: str) -> bool>]

In [22]:
chat = Chat(model, tools=tools)
r = chat.toolloop('Please cancel all orders for customer C1 for me.', trace_func=print)
r

- Retrieving customer C1
Message(id='msg_01D8oJb8DxUWnhFAuvhMbsTp', content=[TextBlock(text="Okay, let's cancel all orders for customer C1:", type='text'), ToolUseBlock(id='toolu_011KAJ7YbLdraqHUyg6DA1ZK', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=In: 537; Out: 72; Total: 609)
- Cancelling order O1
- Cancelling order O2
Message(id='msg_01C9XXzFn66PZnr6nXnrkA5z', content=[TextBlock(text="Based on the customer information, it looks like there are 2 orders for customer C1:\n- Order O1 for Widget A\n- Order O2 for Gadget B\n\nLet's cancel both of these orders:", type='text'), ToolUseBlock(id='toolu_015zF2QPELBh9jzsy19X8u2Y', input={'order_id': 'O1'}, name='cancel_order', type='tool_use'), ToolUseBlock(id='toolu_015Qx73PwE5MWLWaHvhQNGiw', input={'order_id': 'O2'}, name='cancel_order', type='tool_use')], model='claude-3-haiku-20240307', role='as

The cancellation of both orders was successful. All orders for customer C1 have now been cancelled.

<details>

- id: msg_01QFDcsFPMhWahV7efBL9YRd
- content: [{'text': 'The cancellation of both orders was successful. All orders for customer C1 have now been cancelled.', 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 956, 'output_tokens': 25}

</details>

In [24]:
get_customer_info('C1')

- Retrieving customer C1


{'name': 'John Doe',
 'email': 'john@example.com',
 'phone': '123-456-7890',
 'orders': [{'id': 'O1',
   'product': 'Widget A',
   'quantity': 2,
   'price': 19.99,
   'status': 'Cancelled'},
  {'id': 'O2',
   'product': 'Gadget B',
   'quantity': 1,
   'price': 49.99,
   'status': 'Cancelled'}]}

In [25]:
orders

{'O1': {'id': 'O1',
  'product': 'Widget A',
  'quantity': 2,
  'price': 19.99,
  'status': 'Cancelled'},
 'O2': {'id': 'O2',
  'product': 'Gadget B',
  'quantity': 1,
  'price': 49.99,
  'status': 'Cancelled'},
 'O3': {'id': 'O3',
  'product': 'Gadget B',
  'quantity': 2,
  'price': 49.99,
  'status': 'Shipped'}}

In [23]:
chat.use

In: 2239; Out: 243; Total: 2482

In [26]:
chat.use

In: 2239; Out: 243; Total: 2482

In [27]:
chat.toolloop("What is the status of Order 02?")

- Retrieving order O2


The status of Order O2 is 'Cancelled'. This matches the information we saw earlier when retrieving the customer's order details.

So in summary, Order O2 for Gadget B has been successfully cancelled.

<details>

- id: msg_01YRt3MNzcyYjG1hJtAammzh
- content: [{'text': "The status of Order O2 is 'Cancelled'. This matches the information we saw earlier when retrieving the customer's order details.\n\nSo in summary, Order O2 for Gadget B has been successfully cancelled.", 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 1120, 'output_tokens': 52}

</details>

## Need to Watch runaway costs with `Chat` obj, it stores the entire history 

In [28]:
chat.use

In: 4352; Out: 367; Total: 4719

## Code Interpreter

In [29]:
from toolslm.shell import get_shell
from fastcore.meta import delegates
import traceback

In [33]:
@delegates()
class CodeChat(Chat):
    imps = 'os, warnings, time, json, re, math, collections, itertools, functools, dateutil, datetime, string, types, copy, pprint, enum, numbers, decimal, fractions, random, operator, typing, dataclasses'
    def __init__(self, model: Optional[str] = None, ask:bool=True, **kwargs):
        super().__init__(model=model, **kwargs)
        self.ask = ask
        self.tools.append(self.run_cell)
        self.shell = get_shell()
        self.shell.run_cell('import '+self.imps)

In [34]:

@patch
def run_cell(
    self:CodeChat,
    code:str,   # Code to execute in persistent IPython session
): # Result of expression on last line (if exists); '#DECLINED#' if user declines request to execute
    "Asks user for permission, and if provided, executes python `code` using persistent IPython session."
    confirm = f'Press Enter to execute, or enter "n" to skip?\n```\n{code}\n```\n'
    if self.ask and input(confirm): return '#DECLINED#'
    try: res = self.shell.run_cell(code)
    except Exception as e: return traceback.format_exc()
    return res.stdout if res.result is None else res.result

In [35]:
sp = f'''You are a knowledgable assistant. Do not use tools unless needed.
Don't do complex calculations yourself -- use code for them.
The following modules are pre-imported for `run_cell` automatically:

{CodeChat.imps}

Never mention what tools you are using. Note that `run_cell` interpreter state is *persistent* across calls.

If a tool returns `#DECLINED#` report to the user that the attempt was declined and no further progress can be made.'''

In [36]:
def get_user(ignored:str='' # Unused parameter
            ): # Username of current user
    "Get the username of the user running this session"
    return 'Peter'

In [37]:
model = models[-1]

In [38]:
chat = CodeChat(model, tools=[get_user], sp=sp, ask=True)


In [39]:

chat.h = [
    'Calculate the square root of `10332`', 'math.sqrt(10332)',
    '#DECLINED#', 'I am sorry but the request to execute that was declined and no further progress can be made.'
]

In [40]:
def _show_cts(r):
    for o in r.content:
        if hasattr(o,'text'): print(o.text)
        nm = getattr(o, 'name', None)
        if nm=='run_cell': print(o.input['code'])
        elif nm: print(f'{o.name}({o.input})')

In [41]:
def _cont_decline(c):
    return nested_idx(c, 'content', 'content') != '#DECLINED#'

In [42]:
pr = '''Create a 1-line function `checksum` for a string `s`,
that multiplies together the ascii values of each character in `s` using `reduce`.'''
chat.toolloop(pr, temp=0.2, trace_func=_show_cts, cont_func=_cont_decline)

Here is a 1-line function to calculate the checksum of a string using `reduce` and the `ord` function to get the ASCII values:
from functools import reduce

def checksum(s):
    return reduce(lambda x, y: x * ord(y), s, 1)
To test it:
print(checksum("hello"))
The key steps are:

1. Use `reduce` to iteratively multiply the ASCII values of each character.
2. Start with an initial value of 1 for the accumulator.
3. Use a lambda function to multiply the current accumulator by the ASCII value of the current character (`ord(y)`).

This provides a concise one-line implementation of the checksum function.


The key steps are:

1. Use `reduce` to iteratively multiply the ASCII values of each character.
2. Start with an initial value of 1 for the accumulator.
3. Use a lambda function to multiply the current accumulator by the ASCII value of the current character (`ord(y)`).

This provides a concise one-line implementation of the checksum function.

<details>

- id: msg_01SFMrz3Aw8Kc3HvNpqv53m7
- content: [{'text': 'The key steps are:\n\n1. Use `reduce` to iteratively multiply the ASCII values of each character.\n2. Start with an initial value of 1 for the accumulator.\n3. Use a lambda function to multiply the current accumulator by the ASCII value of the current character (`ord(y)`).\n\nThis provides a concise one-line implementation of the checksum function.', 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 882, 'output_tokens': 87}

</details>

In [43]:
pr = 'Use it to get the checksum of the username of this session.'
chat.toolloop(pr, temp=0.2, trace_func=_show_cts)

Okay, let's get the checksum of the username for this session:
get_user({'ignored': 'ignored'})
Now let's calculate the checksum:
from functools import reduce

def checksum(s):
    return reduce(lambda x, y: x * ord(y), s, 1)

print(checksum("Peter"))
The checksum of the username "Peter" is 10791841920.


The checksum of the username "Peter" is 10791841920.

<details>

- id: msg_019bcXoDBqpJcdXfA3ZkuECp
- content: [{'text': 'The checksum of the username "Peter" is 10791841920.', 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 1187, 'output_tokens': 20}

</details>